In [1]:
import pandas as pd
import numpy as np
import math
from math import sin, cos, sqrt, atan2
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import warnings
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
plt.style.use('seaborn')

# warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

In [2]:
df=pd.read_csv('survey.csv')

In [3]:
df.columns=[x.lower() for x in df.columns]

In [4]:
df.drop(list(df[df.self_employed=='Yes'].index),inplace=True)
df.drop('comments',axis=1,inplace=True)

In [5]:
df['timestamp']=[" ".join(x[:-9].split('-')).replace(" ","-") for x in df.timestamp]

In [6]:
conditions=[
    (df.gender.str.lower()=='male')|(df.gender.str.lower()=='m'),
    (df.gender.str.lower()=='female')|(df.gender.str.lower()=="f")|(df.gender.str.lower()=="woman"),
    (df.gender.str.lower()=='mail')|(df.gender.str.lower()=='msle')|(df.gender.str.lower()=='maile')|(df.gender.str.lower()=='mal')
    
]
choises=[
    'Male',
    'Female',
    "Male",
    
]
df['gender']=np.select(conditions,choises,default="Gender Variant")

In [7]:
conditions=[
    df.no_employees=="1-5",            
    df.no_employees=="6-25",         
    df.no_employees=="26-100", 
    df.no_employees=="100-500",       
    df.no_employees=="500-1000" ,           
    df.no_employees=="More than 1000",
]
choises=[
    "[1,5]",
    "[6,25]",
    "[26,100]",
    "[100,500]",
    "[500,1000]",
    "[1000,10000]",
    
]
df['no_employees']=np.select(conditions,choises,default=0)
df["no_employees"]=[eval(x)for x in df.no_employees]

In [8]:
conditions=[
    (df["mental_health_consequence"].str.lower()=='yes'),            
    (df["mental_health_consequence"].str.lower()=='no'),
    (df["mental_health_consequence"].str.lower()=='maybe')|(df["mental_health_consequence"].str.lower()=='don\'t know')|(df["mental_health_consequence"].str.lower()=='not sure'),
]
choises=[
    1,
    0,
    1,

]
df["mental_health_consequence"]=np.select(conditions,choises,default=0)

In [9]:
df

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,2014-08-27,37,Female,United States,IL,NaN,No,Yes,Often,"[6, 25]",No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,0,No,Some of them,Yes,No,Maybe,Yes,No
1,2014-08-27,44,Male,United States,IN,NaN,No,No,Rarely,"[1000, 10000]",No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,1,No,No,No,No,No,Don't know,No
2,2014-08-27,32,Male,Canada,NaN,NaN,No,No,Rarely,"[6, 25]",No,Yes,No,No,No,No,Don't know,Somewhat difficult,0,No,Yes,Yes,Yes,Yes,No,No
3,2014-08-27,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,"[26, 100]",No,Yes,No,Yes,No,No,No,Somewhat difficult,1,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,2014-08-27,31,Male,United States,TX,NaN,No,No,Never,"[100, 500]",Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,0,No,Some of them,Yes,Yes,Yes,Don't know,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12,26,Male,United Kingdom,NaN,No,No,Yes,NaN,"[26, 100]",No,Yes,No,No,No,No,Don't know,Somewhat easy,0,No,Some of them,Some of them,No,No,Don't know,No
1255,2015-09-26,32,Male,United States,IL,No,Yes,Yes,Often,"[26, 100]",Yes,Yes,Yes,Yes,No,No,Yes,Somewhat difficult,0,No,Some of them,Yes,No,No,Yes,No
1256,2015-11-07,34,Male,United States,CA,No,Yes,Yes,Sometimes,"[1000, 10000]",No,Yes,Yes,Yes,No,No,Don't know,Somewhat difficult,1,Yes,No,No,No,No,No,No
1257,2015-11-30,46,Female,United States,NC,No,No,No,NaN,"[100, 500]",Yes,Yes,No,Yes,No,No,Don't know,Don't know,1,No,No,No,No,No,No,No


In [10]:
df.work_interfere=df.work_interfere.fillna("Missing")
df.self_employed=df.self_employed.fillna("Missing")


In [11]:
# name_list=[]
# for i in comments.comments[2].split(' '):
#     name_list.append(i)
# unique_list=list(set(name_list))

In [12]:
# name_list=[]
# for a in comments.comments:
#     for i in a.split(' '):
#         name_list.append(i)
# unique_list=list(set(name_list))
# company_dictionary={}
# for company in unique_list:
#     company_dictionary[company]=0

# for company in name_list:
#     company_dictionary[company]=company_dictionary[company]+1

In [13]:
# for x,y in company_dictionary.items():
#     company_dictionary[x]=list(y)

In [14]:
df

,timestamp,age,gender,country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,2014-08-27,37,Female,United States,IL,Missing,No,Yes,Often,"[6, 25]",No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,0,No,Some of them,Yes,No,Maybe,Yes,No
1,2014-08-27,44,Male,United States,IN,Missing,No,No,Rarely,"[1000, 10000]",No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,1,No,No,No,No,No,Don't know,No
2,2014-08-27,32,Male,Canada,NaN,Missing,No,No,Rarely,"[6, 25]",No,Yes,No,No,No,No,Don't know,Somewhat difficult,0,No,Yes,Yes,Yes,Yes,No,No
3,2014-08-27,31,Male,United Kingdom,NaN,Missing,Yes,Yes,Often,"[26, 100]",No,Yes,No,Yes,No,No,No,Somewhat difficult,1,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,2014-08-27,31,Male,United States,TX,Missing,No,No,Never,"[100, 500]",Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,0,No,Some of them,Yes,Yes,Yes,Don't know,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12,26,Male,United Kingdom,NaN,No,No,Yes,Missing,"[26, 100]",No,Yes,No,No,No,No,Don't know,Somewhat easy,0,No,Some of them,Some of them,No,No,Don't know,No
1255,2015-09-26,32,Male,United States,IL,No,Yes,Yes,Often,"[26, 100]",Yes,Yes,Yes,Yes,No,No,Yes,Somewhat difficult,0,No,Some of them,Yes,No,No,Yes,No
1256,2015-11-07,34,Male,United States,CA,No,Yes,Yes,Sometimes,"[1000, 10000]",No,Yes,Yes,Yes,No,No,Don't know,Somewhat difficult,1,Yes,No,No,No,No,No,No
1257,2015-11-30,46,Female,United States,NC,No,No,No,Missing,"[100, 500]",Yes,Yes,No,Yes,No,No,Don't know,Don't know,1,No,No,No,No,No,No,No


In [15]:
df['country'].value_counts()

United States             695
United Kingdom            157
Canada                     61
Germany                    33
Ireland                    21
Netherlands                20
Australia                  19
France                     11
New Zealand                 8
India                       7
Sweden                      7
Italy                       6
Switzerland                 6
Belgium                     6
Israel                      5
Poland                      5
South Africa                4
Bulgaria                    4
Brazil                      4
Singapore                   4
Russia                      3
Finland                     3
Greece                      2
Mexico                      2
Portugal                    2
Austria                     2
Colombia                    2
Nigeria                     1
Georgia                     1
Slovenia                    1
Norway                      1
Bosnia and Herzegovina      1
Moldova                     1
Philippine

In [16]:
north=['Canada','United States']
uk=['United Kingdom']
eu=['Germany','Ireland','Netherlands','France','Italy','Belgium','Spain','Norway']
for x in df.country.index:
    if df.country[x]in north:
        df.country[x]= "North America"
    elif df.country[x]in uk:
        df.country[x]= "United Kingdom"
    elif df.country[x]in eu:
        df.country[x]= "Europe"
    else:
        df.country[x]= 'Other'

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/carlosruiz/opt

In [17]:
df.country.value_counts()

North America     756
United Kingdom    157
Other             101
Europe             99
Name: country, dtype: int64

In [18]:
north= ['CT', 'DE', 'IL', 'IN', 'IA', 'KS', 'ME', 'MA', 'MD', 'MI', 'MN', 'MO', 'NE','NH', 'NJ', 'NY', 'ND', 'OH', 'PA', 'RI', 'SD', 'VT', 'WI','DC']

south= ['AL', 'AR', 'FL', 'GA', 'KT', 'LA', 'MS', 'NC', 'OK', 'SC', 'TN', 'TX', 'VA', 'WV', 'PR']

west= ['AK', 'AZ', 'CA', 'CO', 'HI', 'ID', 'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY']


for x in df.state.index:
    if df.state[x]in north:
        df.state[x]= "Northern State"
    elif df.state[x]in south:
        df.state[x]= "Southern State"
    elif df.state[x]in west:
        df.state[x]= "Western State"
    else:
        df.state[x]= 'Non US'
        

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [23]:
df.state.value_counts()

Non US            429
Northern State    282
Western State     255
Southern State    147
Name: state, dtype: int64